# CH07-优化模型参数

链接：https://pytorch.org/tutorials/beginner/basics/optimization_tutorial.html

有了模型和数据，接下来就是通过优化参数来训练、验证和测试模型。

训练模型是一个迭代过程；在每次迭代（称为epoch）中：

1. 模型预测输出

2. 计算误差（loss）

3. 计算误差对其参数的导数

4. 使用梯度下降来优化这些参数

重复上述4个步骤，直到loss不再发生变化或epoch次数达到。

## 7.1 前景代码

加载数据集，建立神经网络模型：

In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda

training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=False,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=False,
    transform=ToTensor()
)

train_dataloader = DataLoader(training_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)

class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
            nn.ReLU()
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork()

## 7.2 超参数

定义的超参数有：

+ epoch, 迭代整个数据集的次数

+ batch_size,小批量的样本个数

+ learning rate, 每个epoch/batch 之后，控制模型参数的更新幅度

超参数是可调节的参数，可以控制模型的优化过程。

不同的超参数值会影响模型的训练和收敛速度。

In [3]:
epochs = 5

batch_size = 64

learning_rate = 1e-3

## 7.3 损失函数和优化器

损失函数

损失函数用于衡量预测值与目标值的相近度，训练过程中要最小化损失函数。

常见的损失函数包括: 

+ 用于回归任务的`nn.MSELoss`（均方误差）

+ 用于分类任务的`nn.NLLLoss`（负对数似然）

+ `nn.CrossEntropyLoss`: 结合了`nn.LogSoftmax`和`nn.NLLLoss`



In [4]:
# Initialize the loss function
loss_fn = nn.CrossEntropyLoss()

优化器

优化是在训练步骤中调整模型参数以减少模型误差的过程。

优化算法决定了这个过程是如何执行的。

所有优化逻辑都封装在optimizer对象中。

示例中，使用 SGD(随机梯度下降)优化器。

>PyTorch 中有许多不同的优化器：如 ADAM 和 RMSProp，它们更适用于不同类型的模型和数据。

我们通过传入模型需要训练的参数和学习率来初始化优化器：

In [5]:
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

在训练中，优化分为三步：

+ 调用optimizer.zero_grad()，重置模型参数的梯度（默认情况下渐变相加），为了防止重复计算，我们在每次迭代开始时明确地将它们归零。

+ 调用`.backwards()`进行反向传播（PyTorch 存储了每个参数的损失梯度）。

+ 一旦有了梯度，我们就调用optimizer.step()在反向传递中收集梯度来调整参数。

## 7.4 定义train_loop和test_loop

train_loop用于训练优化模型参数

test_loop用于评估模型的性能

In [6]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        # Compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= size
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

初始化损失函数和优化器，并将其传递给train_loop和test_loop。


In [7]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.308139  [    0/60000]
loss: 2.300107  [ 6400/60000]
loss: 2.290078  [12800/60000]
loss: 2.279682  [19200/60000]
loss: 2.284192  [25600/60000]
loss: 2.262136  [32000/60000]
loss: 2.262522  [38400/60000]
loss: 2.248021  [44800/60000]
loss: 2.239056  [51200/60000]
loss: 2.213274  [57600/60000]
Test Error: 
 Accuracy: 48.3%, Avg loss: 0.034860 

Epoch 2
-------------------------------
loss: 2.237363  [    0/60000]
loss: 2.226662  [ 6400/60000]
loss: 2.177468  [12800/60000]
loss: 2.174523  [19200/60000]
loss: 2.189780  [25600/60000]
loss: 2.131363  [32000/60000]
loss: 2.151162  [38400/60000]
loss: 2.108146  [44800/60000]
loss: 2.118899  [51200/60000]
loss: 2.057504  [57600/60000]
Test Error: 
 Accuracy: 53.9%, Avg loss: 0.032603 

Epoch 3
-------------------------------
loss: 2.121585  [    0/60000]
loss: 2.100652  [ 6400/60000]
loss: 1.992180  [12800/60000]
loss: 1.992150  [19200/60000]
loss: 2.027587  [25600/60000]
loss: 1.918159  [32000/600

链接

[Loss Function](https://pytorch.org/docs/stable/nn.html#loss-functions)

[torch.optim](https://pytorch.org/docs/stable/optim.html)

[warmstarttraining model](https://pytorch.org/tutorials/recipes/recipes/warmstarting_model_using_parameters_from_a_different_model.html)

